In [ ]:
import json
import chess
import zss
import tqdm
import pickle
from main import ChessTreeNode

In [ ]:
trees = {}
with open("all_trees", "r") as f:
    for idx, line in enumerate(tqdm.tqdm(f.readlines())):
        puzzle_id, tree = json.loads(line)
        if len(tree) == 0:
            continue
        trees[puzzle_id] = ChessTreeNode(*tree[0])
        if idx % 100000 == 0:
            with open("tree_dict.pkl", "wb") as f:
                pickle.dump(trees, f)

    with open("tree_dict.pkl", "wb") as f:
        pickle.dump(trees, f)



In [ ]:
line = r'["00Aq4", [[["2k3nr/pppr4/3p3p/5p2/3b2pP/3B2B1/PPPN1PP1/2K1R3 w - - 0 1", "e1e8"], [[["2k1R1nr/pppr4/3p3p/5p2/3b2pP/3B2B1/PPPN1PP1/2K5 b - - 1 1", "d7d8"], [[["2krR1nr/ppp5/3p3p/5p2/3b2pP/3B2B1/PPPN1PP1/2K5 w - - 2 2", "d3f5"], [[["2krR1nr/ppp5/3p3p/5B2/3b2pP/6B1/PPPN1PP1/2K5 b - - 0 2", "c8b8"], [[["1k1rR1nr/ppp5/3p3p/5B2/3b2pP/6B1/PPPN1PP1/2K5 w - - 1 3", "e8d8"], []]]]]]]]]]]] '

want_tree = ChessTreeNode(*json.loads(line)[1][0])
def compare(t1, t2):
    return zss.simple_distance(t1, t2, ChessTreeNode.get_children, ChessTreeNode.get_label, ChessTreeNode.compare)